In [ ]:
SEMINAR_PATH = '/home/jim/PycharmProjects'
import sys
sys.path.append(SEMINAR_PATH)

import RSreader
import RSreader.base.readers as rs
from RSreader.base.bbox import BBox
from RSreader.io import read_raster, align
import RSreader.io as io

import seminar
from seminar.learn.utils import pred_array, GridSearch, simulated_annealing, contingency_distance, train_predict_conf
from seminar.preproc.transformers import FFCombo, TransformerSwitch, InputList, FuncTransformerCombo, DynamicFeatureUnion, FunctionPredictor
from seminar.learn.cluster import cluster_based_classif

import seminar.preproc.readers as rs
from seminar.utils import BiDict, run_jobs
from fiona.crs import from_epsg

import os
import re
import xarray as xr
import pandas as pd
import numpy as np
import random
import rasterio as rio
from functools import partial
import itertools
import networkx as nx
from collections import OrderedDict
import itertools as it

from scipy import signal
import scipy.fftpack as ff
from scipy.fft import fftshift
from scipy.optimize import linear_sum_assignment
import sklearn as skl

import sklearn.cluster as cl
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler, FunctionTransformer, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, adjusted_rand_score, adjusted_mutual_info_score, fowlkes_mallows_score

from sklearn.mixture import GaussianMixture

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

import scipy.optimize
from scipy.optimize import basinhopping

import matplotlib.pyplot as plt
import matplotlib as mpl

import bokeh.palettes as palettes
from bokeh.models import HoverTool

import geoviews as gv
import geoviews.feature as gf
import holoviews as hv

import dill as pickle
from joblib import dump, load
import copy

import hvplot.xarray
import hvplot.pandas
gv.extension('bokeh')

In [ ]:
from importlib import reload
reload(seminar.preproc.transformers)
reload(seminar.learn.utils)
reload(seminar.utils)
reload(seminar.learn.cluster)
reload(scipy)
reload(RSreader.io)

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%load_ext autoreload

## Global Variable Definitions

In [ ]:
arrs_opt_names = ['blue', 'green', 'red', 're1','re2','re3','nir', 're4', 'swir1', 'swir2']
arrs_tex_names = ['vis_rough', 'nir_rough'] + [i + '_' + j for i in ['vis', 'nir'] for j in ['mean', 'var', 'contr', 'hom', 'dis', 'ent', 'm2', 'cor']]
indices = ['EVI', 'mNDWI', 'NDBI', 'NDVI', 'NDWI', 'NDSI', 'UI']

In [ ]:
#all_names = ['vv', 'vh'] + arrs_opt_names + indices + arrs_tex_names
all_names = ['vv', 'vh'] + indices + arrs_tex_names

In [ ]:
n_colors = 100
colorset = list(map(str, range(10))) + ['a', 'b', 'c', 'd', 'e', 'f']
colormap = ['#' + ''.join(map(str, np.random.choice(colorset, 6))) for i in range(n_colors)]

cmap = {i: c for i, c in enumerate(colormap)}
cmap[-1] = '#ffffff'

In [ ]:
colormap = ['#f64e5d', '#fa6d55', '#e7693f', '#8885a9', '#666551', '#f6c55b', 
            '#212a47', '#ebeed1', '#67d6ca', '#118e55', '#8442e4', '#28ba56', 
            '#bc0aca', '#63e1ad', '#05b310', '#0fbccd', '#fb48be', '#cf67e1', 
            '#084c63', '#cee907', '#785044', '#cfde6c', '#9b9d6a', '#9abfb1', 
            '#abd9f1', '#f38902', '#62e138', '#84b069', '#910c57', '#7289de', 
            '#4c1072', '#772b44', '#a23b82', '#e61f62', '#97c9a0', '#b33200', 
            '#c01f7a', '#2c39d5', '#45f65c', '#b0d585', '#1aff26', '#e8eb80', 
            '#3caa57', '#c2f160', '#1fb237', '#71f62c', '#a034b8', '#114789', 
            '#6f941e', '#ec9dea', '#902f73', '#e6aa60', '#ad80f8', '#53bace',
            '#780dcd', '#5a7c18', '#fd22d7', '#4288e5', '#effb3f', '#82cf91', 
            '#4e0c43', '#ea4298', '#d80202', '#d5536b', '#c67615', '#321038', 
            '#5a32ba', '#d7d1f1', '#ec41c3', '#65c7f5', '#1dda6f', '#76682b', 
            '#c07691', '#0cf12d', '#115b78', '#d1f0ee', '#59b790', '#237111', 
            '#aae21d', '#e27639', '#035c61', '#3fae55', '#3f4e22', '#c3accc', 
            '#6788de', '#cf9cd5', '#579ab9', '#313222', '#f6f7c1', '#d88493', 
            '#57c531', '#d0b987', '#f6f930', '#870496', '#9923a1', '#65312c', 
            '#aa0312', '#7fb935', '#d68823', '#33455f']

In [ ]:
ordered_tkwargs_fft = ([{'pre_feature_name': 'vv fr ', 'time_step': 6 / 360}, 
                        {'pre_feature_name': 'vh fr ', 'time_step': 6 / 360}] 
                       #+ [{'pre_feature_name': name + ' fr', 'time_step': 10 / 360} for name in arrs_opt_names]
                        + [{'pre_feature_name': name + ' fr', 'time_step': 10 / 360} for name in indices] \
                        + [{'pre_feature_name': name + ' fr', 'time_step': None} for name in arrs_tex_names]) # this is just for completeness, doesn't make sense with the given temporal resolution

# Readers

In [ ]:
bbox = BBox((5.905e5, 4.835e6, 6.6997e5, 4.798e5), crs=from_epsg(32631))

In [ ]:
# Read VV
# Read in data, crop first, then load from dir with cropped
reader_vv = rs.SeminarReader(time_pattern=r'_([_\-0-9]+)_',
                             incl_pattern='.*',
                             #dirpath='/home/jim/PycharmProjects/seminar/notebooks/data/vv_6d_12d_cropped')
                             dirpath='/home/jim/mount/shared/course/geo441/stud/B_Camargue/s1_crop/vv_dB')
arrs_vv, bboxs_vv = reader_vv.query()
arrs_vv.name = 'vv'
arrs_vv.data = arrs_vv.data.astype(np.float16)


# Read VH
# Read in data, crop first, then load from dir with cropped
reader_vh = rs.SeminarReader(time_pattern=r'_([_\-0-9]+)_',
                             incl_pattern='.*',
                             #dirpath='/home/jim/PycharmProjects/seminar/notebooks/data/vh_6d_12d_cropped')
                             dirpath='/home/jim/mount/shared/course/geo441/stud/B_Camargue/s1_crop/vh_dB')
arrs_vh, bboxs_vh = reader_vh.query()
arrs_vh.name = 'vh'
arrs_vh.data = arrs_vh.data.astype(np.float16)

In [ ]:
# Read Optical
# Beware, this is much data, make sure your RAM is large and empty, ~ 10 gb

reader_opt = rs.SeminarReader(time_pattern=r'_([_\-0-9]+)_',
                             incl_pattern='.*',
                             #dirpath='/home/jim/PycharmProjects/seminar/notebooks/data/vh_6d_12d_cropped')
                             dirpath='/home/jim/mount/shared/course/geo441/stud/B_Camargue/optical/FR_S2_composite_filt_crop')
arrs_opt, bboxs_opt = reader_opt.query(chunks=1000)
arrs_opt.name = 'opt'
arrs_opt.data = (arrs_opt.data * 10000).astype(np.uint16)

In [ ]:
#arrs_opt = arrs_opt.compute()

In [ ]:
# Read Indices
# Read in data, crop first, then load from dir with cropped
arrs_indices = []
readers_ind = []
for index in indices:
    reader = rs.SeminarReader(time_pattern=r'_([_\-0-9]+)_',
                                 incl_pattern='.*',
                                 #dirpath='/home/jim/PycharmProjects/seminar/notebooks/data/vv_6d_12d_cropped')
                                 dirpath='/home/jim/mount/shared/course/geo441/stud/B_Camargue/indices/' 
                                         +'FR_S2_composite_filt_%s_crop' % index)
                                  #dirpath='/home/jim/mount/shared/course/geo441/data/2020_Camargue/optical')
    arrs_ind, _ = reader.query()#chunks=1000)  # bbox=bbox, out=True, out_dir='/home/jim/PycharmProjects/seminar/notebooks/data/test_out')
    arrs_ind.name = index
    arrs_ind.data = arrs_ind.data.astype(np.float16)
    arrs_indices.append(arrs_ind)
    readers_ind.append(reader)
    
#arrs_indices = xr.concat(arrs_indices, 'band')
#arrs_indices.name = 'ind'

In [ ]:
# Read textures
bnds = ['vis', 'nir']
arrs_tex = []
readers_tex = []

for bnd in bnds:
    base_dir = '/home/jim/mount/shared/course/geo441/stud/B_Camargue/optical/FR_S2_textures/%s' % bnd

    reader_tex = rs.SeminarReader(time_pattern=r'_(\d+)_',
                             incl_pattern='_roughness',
                             #dirpath='/home/jim/PycharmProjects/seminar/notebooks/data/vh_6d_12d_cropped')
                             dirpath=base_dir)
    arr_tex, _ = reader_tex.query()
    arrs_tex.append(arr_tex)
    readers_tex.append(reader_tex)

for bnd in bnds:
    base_dir = '/home/jim/mount/shared/course/geo441/stud/B_Camargue/optical/FR_S2_textures/%s' % bnd

    reader_tex = rs.SeminarReader(time_pattern=r'_(\d+)_',
                             incl_pattern='5x5_copy',
                             #dirpath='/home/jim/PycharmProjects/seminar/notebooks/data/vh_6d_12d_cropped')
                             dirpath=base_dir)
    arr_tex, _ = reader_tex.query()
    arrs_tex.append(arr_tex)
    readers_tex.append(reader_tex)

#for bnd in bnds:
#    base_dir = '/home/jim/mount/shared/course/geo441/stud/B_Camargue/optical/FR_S2_textures/%s' % bnd
#
#    reader_tex = rs.SeminarReader(time_pattern=r'_(\d+)_',
#                             incl_pattern='5x5_copy',
#                             #dirpath='/home/jim/PycharmProjects/seminar/notebooks/data/vh_6d_12d_cropped')
#                             dirpath=base_dir)
#    arr_tex, _ = reader_tex.query()
#    arrs_tex.append(arr_tex)
#    readers_tex.append(reader_tex)
    
#for bnd in bnds:
#    base_dir = '/home/jim/mount/shared/course/geo441/stud/B_Camargue/optical/FR_S2_textures/%s' % bnd
#
#    reader_tex = rs.SeminarReader(time_pattern=r'_(\d+)_',
#                             incl_pattern='7x7_copy',
#                             #dirpath='/home/jim/PycharmProjects/seminar/notebooks/data/vh_6d_12d_cropped')
#                             dirpath=base_dir)
#    arr_tex, _ = reader_tex.query()
#    arrs_tex.append(arr_tex)
#    readers_tex.append(reader_tex)
    
arrs_tex = xr.concat(arrs_tex, 'band')
arrs_tex.name = 'tex'

### Ground Truth: Full

In [ ]:
def clean_ground_truth(ground_truth):
    # Bring ground truth in canonical form
    uniques = np.unique(ground_truth)
    mapping = {code: code for val, code in enumerate(uniques)}
    mapping[0] = -1
    mapping[-1] = -1

    mapper = np.vectorize(lambda entry: mapping.get(entry, entry))
    ground_truth.data = mapper(ground_truth.data).astype(np.int16)
    bimapping = BiDict(mapping)
    
    return ground_truth, bimapping

In [ ]:
def construct_class_hierarchy_graph(classes):
        graph = nx.Graph()
        graph.add_node('0')
        recurse_graph_levels('0', graph, 0, classes)
        return graph

def recurse_graph_levels(parent_node, graph, new_layer, subset_classes):
    new_nodes = set([str(cls_out)[new_layer] for cls_out in subset_classes if new_layer < len(str(cls_out))])

    if new_nodes != set():
        for new_node in new_nodes:
            new_subset = filter(lambda x: x.startswith(parent_node[1:] + new_node), subset_classes)
            recurse_graph_levels(parent_node + new_node, graph, new_layer + 1, new_subset)
            graph.add_node(parent_node + new_node)
            graph.add_edge(parent_node, parent_node + new_node)

In [ ]:
# Read ground truth and align
ground_truth_classif, bboxs = io.read_raster('/home/jim/Dropbox/RS-Camargue-Box/Klassierung_2/pnrc_mos_2016_c_rasterized30x30m_UTM31N.tif')
ground_truth_classif.data = ground_truth_classif.astype(rio.int32)

aligned_gt = align(arrs_vv, ground_truth_classif,
                   '/home/jim/PycharmProjects/seminar/notebooks/data/aligned_ground_truth.tif')

In [ ]:
aligned_gt, bimapping = clean_ground_truth(ground_truth=aligned_gt)
gt = pd.DataFrame(aligned_gt.data.flatten()).copy()

In [ ]:
valid_gt = np.where(gt > -1)
only_valid_gt = gt.iloc[valid_gt[0]]

### Ground Truth: Reduced

In [ ]:
ground_truth_classif, bboxs = io.read_raster('/home/jim/mount/shared/course/geo441/stud/B_Camargue/alternativ_pnrc_classes/pnrc_2016_DGL_mod.tif')

ground_truth_classif.data = ground_truth_classif.astype(rio.int32)
aligned_gt_reduced = align(arrs_vv, ground_truth_classif, '/home/jim/PycharmProjects/seminar/notebooks/data/aligned_ground_truth_reduced.tif')

In [ ]:
aligned_gt_reduced, bimapping_reduced = clean_ground_truth(ground_truth=aligned_gt_reduced)
gt_reduced = pd.DataFrame(aligned_gt_reduced.data.flatten()).copy()

In [ ]:
valid_gt = np.where(gt_reduced > -1)
only_valid_gt_reduced = gt_reduced.iloc[valid_gt[0]]

In [ ]:
aligned_gt.hvplot.image(width=1200, height=1200)

### Water masks

In [ ]:
reader_wm = rs.SeminarReader(time_pattern=r'_([_\-0-9]+)_',
                             incl_pattern='.*',
                             #dirpath='/home/jim/PycharmProjects/seminar/notebooks/data/vh_6d_12d_cropped')
                             dirpath='/home/jim/mount/shared/course/geo441/stud/B_Camargue/watermask/S1_vh_watermask/')
arrs_wm, bboxs_wm = reader_wm.query(chunks=1000)

In [ ]:
arr_wm_all, _ = io.read_raster('/home/jim/mount/shared/course/geo441/stud/B_Camargue/watermask/S1_vh_watermask_extent/S1IWGRDH_VH_watermask_maxExtent.tif')

In [ ]:
reader_wm_yearly_max = rs.SeminarReader(time_pattern=r'_([_\-0-9]+)',
                             incl_pattern='_maxExtent_',
                             #dirpath='/home/jim/PycharmProjects/seminar/notebooks/data/vh_6d_12d_cropped')
                             dirpath='/home/jim/mount/shared/course/geo441/stud/B_Camargue/watermask/S1_vh_watermask_extent/yearly/')
arrs_wm_yearly_max, bboxs_wm = reader_wm_yearly_max.query()

In [ ]:
reader_wm_yearly_min = rs.SeminarReader(time_pattern=r'_([_\-0-9]+)',
                             incl_pattern='_maxExtent_',
                             #dirpath='/home/jim/PycharmProjects/seminar/notebooks/data/vh_6d_12d_cropped')
                             dirpath='/home/jim/mount/shared/course/geo441/stud/B_Camargue/watermask/S1_vh_watermask_extent/yearly/')
arrs_wm_yearly_min, bboxs_wm = reader_wm_yearly_min.query()

### to pandas

#### VV

In [ ]:
xx_unnorm = arrs_vv.data.reshape(arrs_vv.shape[0], -1).transpose()
only_valid_vv = pd.DataFrame(xx_unnorm, columns=arrs_vv.time.data)

#### VH

In [ ]:
xx_unnorm = arrs_vh.data.reshape(arrs_vv.shape[0], -1).transpose()
only_valid_vh = pd.DataFrame(xx_unnorm, columns=arrs_vh.time.data)

#### INDICES

In [ ]:
xx_unnorm = [ind.data.reshape(ind.shape[0], -1).transpose() for ind in arrs_indices]
only_valid_ind = [pd.DataFrame(ind, columns=arrs_indices[i].time.data) for i, ind in enumerate(xx_unnorm)]

#### OPTICAL

In [ ]:
#xx_unnorm = [arrs_opt[:, i, ...].data.reshape(arrs_opt.shape[0], -1).transpose() for i in range(arrs_opt.shape[1])]
#only_valid_opt = [pd.DataFrame(opt, columns=arrs_opt[:, i].time.data) for i, opt in enumerate(xx_unnorm)]

#### TEXTURE

In [ ]:
xx_unnorm = [arrs_tex[:, i, ...].data.reshape(arrs_tex.shape[0], -1).transpose() for i in range(arrs_tex.shape[1])]
only_valid_tex = [pd.DataFrame(tex, columns=arrs_tex[:, i].time.data) for i, tex in enumerate(xx_unnorm)]

### sample

In [ ]:
def filter_time(X, year):
    if type(X) is InputList:
        in_cols = [[i for i, col in enumerate(X.get(name).columns) if col.year == year] for name in X.names]
        return InputList(OrderedDict([(X.names[i], x.iloc[:, in_cols[i]]) for i, x in enumerate(X.list)]))
    else:
        in_cols = [[i for i, col in enumerate(X[name].columns) if col.year == year] for name in X.keys()]
        return OrderedDict([(key, x.iloc[:, in_cols[i]]) for i, (key, x) in enumerate(X.items())])


In [ ]:
def mask_dframe_with_dframe(arr, mask):
    if type(arr) is InputList:
        inds = mask.iloc[np.where(mask == 0)[0]].index
        return InputList(OrderedDict([(arr.names[key], val.drop(inds, errors='ignore')) for key, val in arr.map.items() if val is not None]))
    else:
        return arr.drop(mask.iloc[np.where(mask == 0)[0]].index, errors='ignore')

In [ ]:
# sample from dataset
def sample(n, ground_truth, mask=None):
    if type(ground_truth) is list:
        if mask is not None:
            ground_truth = [mask_dframe_with_dframe(gt, mask) for gt in ground_truth]
        intersect = ground_truth[0].index.intersection(ground_truth[1].index)
        for i in range(2, len(ground_truth)):
            intersect = intersect.intersection(ground_truth[i].index)
        ground_truth = ground_truth[0].loc[intersect]
    else:
        if mask is not None:
            ground_truth = mask_dframe_with_dframe(ground_truth, mask)

    sample_inds = ground_truth.sample(n=n)
    samples = OrderedDict([(name, df.loc[sample_inds.index, :]) for name, df in only_valid_dfxx_unnorm.items()])
    
    gt_samples = ground_truth.loc[sample_inds.index]
    return samples, gt_samples, sample_inds.index

In [ ]:
def reduce_gt(gt, depth_wetland, depth_nonwetland):
    mapper = np.vectorize(lambda entry, dw, dnw: str(bimapping.inverse.get(entry)[-1])[:dw ]  if entry >= 44  else str(bimapping.inverse.get(entry)[-1])[:dnw ] )
    gtt = mapper(gt, depth_wetland, depth_nonwetland).astype(np.int16)
    
    # make invalids take same number
    gtt[np.where(gtt == 0)] = -1
    
    return gtt

In [ ]:
full_dfxx_unnorm = OrderedDict([('vv', only_valid_vv), ('vh', only_valid_vh)])
full_dfxx_unnorm.update(OrderedDict([(name, ind) for name, ind in zip(indices, only_valid_ind)]))
full_dfxx_unnorm.update(OrderedDict([(name, tex) for name, tex in zip(arrs_tex_names, only_valid_tex)]))

only_valid_dfxx_unnorm = full_dfxx_unnorm

In [ ]:
samples, gt_samples, sample_index = sample(1000, ground_truth=only_valid_gt)

## Define different Transformers

In [ ]:
def norm(pixs, with_cloud_mask=False):
    ret = OrderedDict()

    # s1 bands
    rang = np.array([-20, 5])
    for name, pix in pixs.get('vv', 'vh', outtyp='dict').items(): #pixs.get(slice(None, 2)):
        if pix is not None:
            ret_pix = pix.copy()
            ret_pix = np.clip((ret_pix - rang.mean()) / (rang[1] - rang[0]), -1, 1)
            ret[name] = ret_pix
        
    # opt bands
    rang = np.array([0, 65520])
    for name, pix in pixs.get(*arrs_opt_names, outtyp='dict').items(): #pixs.get(slice(2, 2+len(arrs_opt_names))):
        if pix is not None:
            ret_pix = pix.copy().astype(np.float32)
            ret_pix = (ret_pix - rang.mean()).astype(np.float32) / (rang[1] - rang[0])
            ret[name] = ret_pix
        
    # index bands
    rang = np.array([-10000, 10000])
    for name, pix in pixs.get(*indices, outtyp='dict').items(): #pixs.get(slice(2+len(arrs_opt_names), None)):
        if pix is not None:
            ret_pix = pix.copy()
            ret_pix = np.clip((ret_pix - rang.mean()) / (rang[1] - rang[0]), -1, 1)
            ret[name] = ret_pix
        
    # tex bands
    rang = np.array([-100, 100])
    for name, pix in pixs.get(*arrs_tex_names, outtyp='dict').items(): #pixs.get(slice(2+len(arrs_opt_names), None)):
        if pix is not None:
            ret_pix = pix.copy()
            ret_pix = np.clip((ret_pix - rang.mean()) / (rang[1] - rang[0]), -1, 1)
            ret[name] = ret_pix
    
    return InputList(ret)

In [ ]:
normer = FunctionTransformer(norm)

In [ ]:
std_norm = FunctionTransformer(func=lambda X: np.einsum('ij, i -> ij', X - X.mean(axis=1)[:, None], 
                                                        1 / (X.std(axis=1) + 1e-2)))

In [ ]:
def reshape(X):
    if hasattr(X, 'values'):
        X = X.values
    return X.reshape(-1, 1)

In [ ]:
def get_scaled_func_transformer(func, *fargs, **fkwargs):
    pip = Pipeline([#('scale', std_norm),#RobustScaler(quantile_range=quantile_range)),
                    #('stop_inf', stop_inf), 
                    ('func', FunctionTransformer(lambda X: func(X, *fargs, **fkwargs))),
                    ('reshape', FunctionTransformer(reshape)),
                    #('scale', RobustScaler(quantile_range=quantile_range))])
                    ('scale_out', StandardScaler()), 
                    ('clip', FunctionTransformer(lambda x: np.clip(x, -3, 3))),])
    return pip

In [ ]:
def diff_suwi(X, reducer=np.mean, *args, **kwargs):
    su_cols = [i for i, c in enumerate(X.columns) if c.month >= 6 and  c.month <= 9]
    wi_cols = [i for i, c in enumerate(X.columns) if c.month <= 2 or  c.month >= 11]
    
    if len(su_cols) > 0 and len(wi_cols) > 0:
        return reducer(X.iloc[:, su_cols], axis=1) - reducer(X.iloc[:, wi_cols], axis=1)
    else:
        return np.zeros((X.shape[0], 1))

## Train Plain

In [ ]:
#is_concerned=[True, False] + [False, False, False] + [False] * (len(arrs_opt_names) - 3) + [False] * len(indices)
is_concerned=[True, False] + [False] * len(indices) + [False] * len(arrs_tex_names)

In [ ]:
pip_plain = Pipeline([('scale', normer),
                      ('preproc', FFCombo(is_concerned=is_concerned, len_inp=len(is_concerned),
                                          scaler=std_norm, thr_freq=5, scale=True, quantile_range=(0.25, 0.75), 
                                          ordered_tkwargs=ordered_tkwargs_fft)),
                      ('clustering', cl.KMeans(n_clusters=8, n_jobs=12))])

In [ ]:
_ = pip_plain.fit(InputList(samples))

In [ ]:
a = pip_plain['clustering'].cluster_centers_
inds = pip_plain['preproc'].get_feature_names()
pd.DataFrame(a.transpose(), index=inds).hvplot(rot=90, width=1200, height=400, cmap=colormap[:pip_plain['clustering'].n_clusters]) #, xformatter='%.3f')#.opts(colorbar=True, cmap='paired')

In [ ]:
arr, pred = pred_array(model=pip_plain, inp=InputList(only_valid_dfxx_unnorm), model_arr=arrs_vv[0, 0], n_batches=400, n_jobs=3)

In [ ]:
arr.hvplot.image('x', 'y', width=900, height=900, rasterize=False).opts(colorbar=True, cmap=['#ffffff'] + colormap[:pip_plain['clustering'].n_clusters])

In [ ]:
a = [np.average(only_valid_dfxx_unnorm.loc[pred.index[np.where(pred == cls_ind)[0]], :], axis=0) 
     for cls_ind in range(pip['clustering'].n_clusters)]
means = np.stack(a)

In [ ]:
pd.DataFrame(means.transpose()).hvplot()

## Train Extended

In [ ]:
samples, gt_samples, sample_index = sample(10000, ground_truth=only_valid_gt)

In [ ]:
#is_concerned = [False, True] + [False] * len(arrs_opt_names) + [False] * len(indices)
#is_concerned_ext = [True, True] + [True, True, True] + [False]* (len(arrs_opt_names) - 3) + [True] * len(indices)

is_concerned = [False, True] + [False] * len(indices) + [False] * len(arrs_tex_names)
is_concerned_ext = [True, True] + [True] * len(indices) + [True] * len(arrs_tex_names)

In [ ]:
pip_fft = Pipeline([('preproc', FFCombo(len_inp=len(samples), is_concerned=is_concerned,
                                              scaler=std_norm, thr_freq=5, scale=True, quantile_range=(0.25, 0.75), 
                                              ordered_tkwargs=ordered_tkwargs_fft))
                   ])

In [ ]:
pip_other = Pipeline([('preproc', 
                       FeatureUnion([
                           #('mean', FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                           #                                       transformer=get_scaled_func_transformer(np.mean, axis=1))),
                           ('diff_suwi_mean', FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                  transformer=get_scaled_func_transformer(diff_suwi))),]))
                           #('std', FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), #edge_cols=edges,
                           #                                   transformer=get_scaled_func_transformer(np.en, axis=1))),
                           #('max', FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples),
                           #                             transformer=get_scaled_func_transformer(np.max, axis=1)))]))
                     ])


In [ ]:
preproc = Pipeline([('scale', normer),
                    ('feats', FeatureUnion([('scaled_fft', pip_fft),
                                            ('scaled_ext', pip_other)]))
                   ])

In [ ]:
pip_ext = Pipeline([('preproc', preproc),
                    #('clustering', cl.KMeans(n_clusters=10, n_jobs=12)),
                    ('clustering', TransformerSwitch(transformers=[cl.KMeans(n_clusters=100, n_jobs=12),
                                                                   cl.AffinityPropagation(max_iter=500),
                                                                   cl.AgglomerativeClustering(n_clusters=10, linkage='ward'),
                                                                   GaussianMixture(n_components=10)
                                                                  ], is_on=-1))])

In [ ]:
_ = pip_ext.fit(InputList(samples))

In [ ]:
#a = pip_ext['clustering'].get().cluster_centers_
a = pip_ext['clustering'].get().means_

inds = (pip_fft['preproc'].get_feature_names() + \
              #['mean %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i]] + \
              ['diff_suwi %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i]])# + \
              #['max %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i]])
pd.DataFrame(a.transpose(), index=inds).hvplot(rot=90, width=1200, height=400, cmap=colormap) #cmap=['#ffffff'] + palettes.Category10[8]) #, xformatter='%.3f')#.opts(colorbar=True, cmap='paired')

In [ ]:
inp = mask_dframe_with_dframe(InputList(only_valid_dfxx_unnorm), pd.DataFrame(arr_wm_all.data.flatten()))
arr, pred = pred_array(model=pip_ext, inp=inp, model_arr=arrs_vv[0, 0], n_batches=500, n_jobs=5, verbose=30)

In [ ]:
arr.hvplot.image('x', 'y', width=900, height=900, 
                               rasterize=False).opts(colorbar=True, cmap=colormap[:10]) #palettes.Category10[8]) #.opts(color_levels=9, colorbar=True, cmap='paired') 

In [ ]:
a = [np.average(only_valid_dfxx_unnorm.loc[pred.index[np.where(pred == cls_ind)[0]], :], axis=0) 
     for cls_ind in range(pip['clustering'].n_clusters)]
means = np.stack(a)

In [ ]:
only_valid_dfxx_unnorm.values.dtype

In [ ]:
pd.DataFrame(means.transpose()).hvplot()

# Validation

## Look at Confusion Matrix

In [ ]:
ground_truth_depth_non_wetland = 2
ground_truth_depth_wetland = 2

In [ ]:
samples, gt_samples, sample_index = sample(50000, ground_truth=only_valid_gt)

In [ ]:
samples, gt_samples, sample_index = sample(50000, ground_truth=only_valid_gt, mask=pd.DataFrame(arr_wm_all.data.flatten()))
samples = filter_time(samples, year=2017)

### Prepare ground truth

In [ ]:
gtt_samples = reduce_gt(gt_samples, ground_truth_depth_wetland, ground_truth_depth_non_wetland)

In [ ]:
gtr = reduce_gt(aligned_gt.data, ground_truth_depth_wetland, ground_truth_depth_non_wetland)

In [ ]:
gtr = xr.DataArray(gtr, dims=['band', 'y', 'x'])
gtr.attrs = aligned_gt.attrs
gtr = gtr.assign_coords(aligned_gt.coords)

### Definitions

In [ ]:
#is_concerned = [False, True] + [False] * len(arrs_opt_names) + [False] * len(indices)
#is_concerned_ext = [True, True] + [True, True, True] + [False]* (len(arrs_opt_names) - 3) + [True] * len(indices)
is_conc_fft_vv_vh = [True, True] + [False] * len(indices) + [False] * len(arrs_tex_names)
is_conc_fft_vv_vh_evi = [True, True] + [True, False, False, False, False, False, False] + [False] * len(arrs_tex_names)
is_conc_fft_vh = [False, True] + [False] * len(indices) + [False] * len(arrs_tex_names)
is_conc_fft_vv = [True, False] + [False] * len(indices) + [False] * len(arrs_tex_names)
is_conc_fft_evi = [False, False, True] + [False] * (len(indices) - 1) + [False] * len(arrs_tex_names)
is_conc_fft_none = [False, False, False] + [False] * (len(indices) - 1) + [False] * len(arrs_tex_names)

is_concerned = is_conc_fft_vv_vh_evi
is_concerned_ext = [True, True] + [True] * len(indices) + [True] * len(arrs_tex_names)

In [ ]:
pip_fft = Pipeline([('fft', FFCombo(len_inp=len(samples), is_concerned=is_concerned,
                                              scaler=std_norm, thr_freq=5, scale=True, quantile_range=(0.25, 0.75), 
                                              ordered_tkwargs=ordered_tkwargs_fft))
                   ])

In [ ]:
pip_other = Pipeline([('preproc', 
                       DynamicFeatureUnion([
                           ('min', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(np.min, axis=1))], is_on=1)),
                            
                           ('max', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(np.max, axis=1))], is_on=1)),
                           
                           ('mean', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(np.median, axis=1))], is_on=1)),
                           ('q25', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(partial(np.quantile, q=0.25), axis=1))], is_on=1)),
                           ('q75', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(partial(np.quantile, q=0.75), axis=1))], is_on=1)),
                           ('std', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(np.std, axis=1))], is_on=1)),
                           ('median', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(np.mean, axis=1))], is_on=1)),
                           ('diff_suwi_mean', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(diff_suwi, reducer=np.mean))], is_on=1)),
                           ('diff_suwi_std', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(diff_suwi, reducer=np.std))], is_on=1)),
                           ('diff_suwi_q75', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(diff_suwi, reducer=partial(np.quantile, q=0.75)))], is_on=1)),
                           ('diff_suwi_q25', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(diff_suwi, reducer=partial(np.quantile, q=0.25)))], is_on=1))
                     ]))])


In [ ]:
preproc = Pipeline([#('input_format', FunctionTransformer(to_input_list)), 
                    ('scale', normer),
                    #('pca', TransformerSwitch(transformers=[None,
                    #                                        PCA(n_components=40)], is_on=0)),
                    #('feats', FeatureUnion([('scaled_fft', pip_fft),
                    #                       ('scaled_ext', pip_other)]))
                    ('feats', pip_other)
                   ])

In [ ]:
pip_ext = Pipeline([('preproc', preproc),
                    ('pca', TransformerSwitch(transformers=[None, PCA(n_components=10)], is_on=False, transparent=True)), 
                    ('clustering', TransformerSwitch(transformers=[cl.KMeans(n_clusters=30, n_jobs=12),
                                                                   #cl.AffinityPropagation(max_iter=500),
                                                                   GaussianMixture(n_components=150,),
                                                                   #cl.AgglomerativeClustering(n_clusters=10, linkage='ward'),
                                                                   #cl.MeanShift(max_iter=500, n_jobs=12)
                                                                  ], is_on=0))])

In [ ]:
_ = pip_ext.fit(InputList(samples))

In [ ]:
a = pip_ext['clustering'].get().cluster_centers_
#a = pip_ext['clustering'].get().means_

inds = (#pip_fft['fft'].get_feature_names() + \
              ['mean %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['mean'] is not None] + 
              ['min %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['min'] is not None] + 
              ['max %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['max'] is not None] + 
              ['median %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i]] + \
              ['q25 %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['q25'] is not None] + 
              ['q75 %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['q75'] is not None] + 
              ['std %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['std'] is not None] +
              ['diff_suwi_mean %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['diff_suwi_mean'] is not None] +
              ['diff_suwi_std %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['diff_suwi_std'] is not None] +
              ['diff_suwi_q75 %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['diff_suwi_q75'] is not None] +
              ['diff_suwi_q25 %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['diff_suwi_q25'] is not None]
       )
pd.DataFrame(a.transpose(), index=inds).hvplot(rot=90, width=1700, height=700, cmap=colormap) #cmap=['#ffffff'] + palettes.Category10[8]) #, xformatter='%.3f')#.opts(colorbar=True, cmap='paired')

### Fit & Predict

In [ ]:
pip = pip_ext
clf, (valids, conf) = train_predict_conf(model=pip, 
                                          #dset=InputList(only_valid_dfxx_unnorm), 
                                          dset=mask_dframe_with_dframe(InputList(only_valid_dfxx_unnorm), pd.DataFrame(arr_wm_all.data.flatten())),
                                          model_arr=arrs_vv[0, :1], 
                                          ground_truth=gtr,
                                          classif_out_path='/home/jim/PycharmProjects/seminar/notebooks/data/clustering.tif',
                                          samples=InputList(samples),
                                          is_aligned=True,
                                          load_classif=False,
                                          gt=None,
                                          train=False,
                                          gt_nan=-1,
                                          pred_nan=-1, 
                                          n_jobs=5, chunks=1, verbose=10, n_batches=300, joblib=True)

In [ ]:
gtr_labels = np.unique(gtr.data[valids])
cluster_labels = np.unique(clf.data[valids])
labels = np.r_[cluster_labels, gtr_labels]

In [ ]:
#n_clusters = pip_ext['clustering'].get().n_components #len(pip_ext['clustering'].get().cluster_centers_)
n_clusters = pip_ext['clustering'].get().n_clusters

In [ ]:
n_classes = len(gtr_labels)

In [ ]:
clf.hvplot.image('x', 'y', width=1200, height=1200, 
                               rasterize=False).opts(colorbar=True, cmap=colormap[:n_clusters])

## Assignment: maximize recall

### Hungarian Method (one-to-one)

In [ ]:
ax3 = plt.subplot(1, 2, 1)
ax3.matshow(np.log(conf[n_clusters:, :n_clusters] + 1))
ax3.set_title('Unordered Clustering', pad=30)

# sort with Hungarian method (*note* this is a bijective mapping BAD!!)
row_ind, col_ind = linear_sum_assignment(-conf)
ax2 = plt.subplot(1, 2, 2)
ax2.matshow(np.log(conf + 1)[row_ind, :][:, col_ind][n_clusters:, n_clusters:])
ax2.set_title('Best Bijective Map', pad=25)


plt.gcf().set_size_inches((30,15))

In [ ]:
hungarian_assignment = {cluster: labels[col_ind[cluster]] for cluster in np.where(col_ind >= n_clusters)[0]}

In [ ]:
print(hungarian_assignment)

In [ ]:
sorted_clf = clf.copy()

uniques = np.unique(list(hungarian_assignment.values()))
idx_map = dict([(j, i) for i, j in enumerate(uniques)])

mapper = np.vectorize(lambda cluster: idx_map.get(hungarian_assignment.get(cluster, -1), -1))

sorted_clf.data = mapper(clf.data)

In [ ]:
sorted_clf.hvplot.image('x', 'y', rasterize=False, height=1200, width=1200, cmap=['#ffffff'] + colormap[:n_classes])# cmap='viridis', # cmap=['#ffffff'] + palettes.Category10[9])

### Max Assignment (max mean precision)

In [ ]:
label_set, label_counts = np.unique(gtr.data, return_counts=True)

label_to_idx = dict(zip(label_set, range(len(label_set))))
label_to_idx[-1] = -1
mapper_gtr = np.vectorize(lambda ind: label_to_idx.get(ind))

In [ ]:
# max sorting (maximizing recall)
max_assignment = labels[np.argmax(conf, axis=0)][:n_clusters]
max_assignment

In [ ]:
sorted_clf = clf.copy()
uniques = np.unique(max_assignment)
idx_map = dict([(j, i) for i, j in enumerate(uniques)])

mapper = np.vectorize(lambda cluster: label_to_idx.get(max_assignment[cluster], -1))
sorted_clf.data = mapper(sorted_clf.data)

sorted_gtr = gtr.copy()
sorted_gtr.data = mapper_gtr(sorted_gtr.data)


In [ ]:
sorted_clf.hvplot.image('x', 'y', rasterize=False, height=1200, width=1200, 
                        cmap=colormap[:len(set(sorted_clf.data.flatten())) + 1])# cmap='viridis', # cmap=['#ffffff'] + palettes.Category10[9])

## General Procedure: minimize contingency metric over cluster-label assignments

In [ ]:
label_set, label_counts = np.unique(gtr.data, return_counts=True)

# take out -1
valid_idx = np.where(label_set != -1)
label_set = label_set[valid_idx]
label_counts = label_counts[valid_idx]

argsort = np.argsort(label_set)
label_set = label_set[argsort]
label_counts = label_counts[argsort]

label_weights = label_counts / np.sum(label_counts)
print(dict(zip(label_set, label_weights)))

In [ ]:
#inds = np.random.choice(range(len(valids[0])), 10000)
#inds = list(zip(*[(valids[0][i], valids[1][i], valids[2][i]) for i in inds]))

calc_contingency_dist = partial(contingency_distance, score=partial(f1_score, average=None), 
                                ground_truth=gtr.data.flatten(), clustering=clf.data.flatten(), trn_inds=sample_index) # valid_pix=valids, 

In [ ]:
x0 = max_assignment[:n_clusters].copy()
assign, hist, T_end, samples = simulated_annealing(x0, calc_contingency_dist, 
                                   label_set=label_set, label_weights=label_weights, epochs=100, T=5e-3, eta=0.96, max_change=1, verbose=True, sample=10, subset=0.7)

In [ ]:
plt.plot([h[1].mean() for h in hist])

In [ ]:
lab, res = contingency_distance(prev_perm=x0, score=partial(f1_score, average=None), ground_truth=gtr.data, valid_pix=valids, clustering=clf.data)

In [ ]:
assign_200_full = assign.copy()

In [ ]:
res_200_full = {l: 1 - r for l, r in zip(lab, res)}

In [ ]:
pd.DataFrame(res_200_full, index=[0]).hvplot.bar(xlabel='class', ylabel='f1')

In [ ]:
assignment_to_label = dict(zip(range(len(assign)), assign))
assignment_to_label[-1] = -1  

mapper = np.vectorize(lambda ind: label_to_idx.get(assignment_to_label.get(ind)))

In [ ]:
sorted_clf = clf.copy()
sorted_clf.data = mapper(clf.data)

sorted_gtr = gtr.copy()
sorted_gtr.data = mapper_gtr(sorted_gtr.data)

sorted_dset = xr.Dataset({'gtr':sorted_gtr, 'clf':sorted_clf})


In [ ]:
fig = hv.Image(sorted_clf, kdims=['x', 'y'],).opts(height=900, width=900, cmap=colormap, colorbar=True) + hv.Image(sorted_gtr, kdims=['x', 'y'],).opts(height=900, width=900, cmap=colormap, colorbar=True)

In [ ]:
fig

In [ ]:
sorted_dset.hvplot.image('x', 'y', ['clf', 'gtr'], height=1200, width=1200, cmap=colormap, colorbar=True)

### Show new Confusion Matrix

In [ ]:
def get_reordered(assignment, n_clusters):
    assign_argsort = np.argsort(assignment[:n_clusters])
    x_ticks = np.where(np.diff(assignment[assign_argsort]) > 0)[0]
    x_ticks_labels = assignment[assign_argsort][x_ticks]
    return (x_ticks, x_ticks_labels), assign_argsort

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import colors

%matplotlib inline
def plot_conf(conf, n_clusters, n_classes, labels):
    n_classes = len(np.unique(gtr))

    (xticks_assign, xticks_labels_assign), assign_argsort = get_reordered(assign, n_clusters)
    conf_reordered_assign = conf[n_clusters:, assign_argsort]

    #(xticks_maxassign, xticks_labels_maxassign), maxassign_argsort = get_reordered(max_assignment, n_clusters)
    #conf_reordered_maxassign = conf [n_clusters:, maxassign_argsort]


    ax2 = plt.subplot(3, 1, 1)
    im = ax2.matshow(conf[n_clusters:, :n_clusters], norm=colors.SymLogNorm(vmin=0, vmax=np.max(conf), linthresh=10, base=10), )
    ax2.set_title('Unordered Confusion Matrix', pad=30)

    divider = make_axes_locatable(ax2)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    h = plt.gcf().colorbar(im, cax=cax, orientation='vertical')
    
    
    
    ax1 = plt.subplot(3, 1, 2)
    ax1.matshow(conf_reordered_assign, norm=colors.SymLogNorm(vmin=0, vmax=np.max(conf), linthresh=10, base=10), )
    
    ax1.set_xticks(ticks=xticks_assign + 0.5)
    ax1.set_xticklabels(labels=xticks_labels_assign, rotation=-90)
    
    ax1.set_yticks(ticks=np.arange(n_classes) + 0.5, )
    ax1.set_yticklabels(labels=labels)

    plt.vlines(xticks_assign + 0.5, -0.5, conf.shape[0] - n_clusters - 0.5, 'red')
    ax1.set_title('Locally Optimal Cluster-Class Map', pad=30)

    divider = make_axes_locatable(ax1)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    plt.gcf().colorbar(im, cax=cax, orientation='vertical')


    plt.gcf().set_size_inches((25,10))
    plt.tight_layout()
    
    """
    ax3 = plt.subplot(3, 1, 2)
    ax3.matshow(conf_reordered_maxassign, norm=colors.SymLogNorm(vmin=0, vmax=np.max(conf), linthresh=10), )

    plt.xticks(ticks=xticks_maxassign + 0.5, labels=xticks_labels_maxassign, rotation=-90)
    plt.yticks(ticks=np.arange(n_classes) + 0.5, labels=labels[n_clusters:])

    plt.vlines(xticks_maxassign + 0.5, -0.5, conf.shape[0] - n_clusters - 0.5, 'red')
    ax3.set_title('Initial Cluster-Class Map', pad=30)

    divider = make_axes_locatable(ax3)
    cax = divider.append_axes('right', size='1%', pad=0.05)
    plt.gcf().colorbar(im, cax=cax, orientation='vertical')
    """

## Grid search

In [ ]:
only_valid_gt.index.intersection(only_valid_gt_reduced.index)

In [ ]:
samples, gt_samples, sample_index = sample(100000, ground_truth=[only_valid_gt, only_valid_gt_reduced], mask=pd.DataFrame(arr_wm_all.data.flatten()))
gt_simple = only_valid_gt_reduced.loc[sample_index]

samples  = filter_time(samples, year=year)

gt_red = reduce_gt(gt_samples, depth_nonwetland=2, depth_wetland=2)
gt_red1 = reduce_gt(gt_samples, depth_nonwetland=2, depth_wetland=3)
gt_red2 = reduce_gt(gt_samples, depth_nonwetland=2, depth_wetland=4)
gt_red3 = reduce_gt(gt_samples, depth_nonwetland=3, depth_wetland=3)
gt_red4 = reduce_gt(gt_samples, depth_nonwetland=3, depth_wetland=4)
gt_red5 = reduce_gt(gt_samples, depth_nonwetland=4, depth_wetland=4)
gts = [gt_simple.values, gt_red, gt_red1, gt_red2, gt_red3, gt_red4, gt_red5]

### prepare data

In [ ]:
#is_concerned = [False, True] + [False] * len(arrs_opt_names) + [False] * len(indices)
#is_concerned_ext = [True, True] + [True, True, True] + [False]* (len(arrs_opt_names) - 3) + [True] * len(indices)
is_conc_fft_vv_vh = [True, True] + [False] * len(indices) + [False] * len(arrs_tex_names)
is_conc_fft_vv_vh_evi = [True, True] + [True, False, False, False, False, False, False] + [False] * len(arrs_tex_names)
is_conc_fft_vh = [False, True] + [False] * len(indices) + [False] * len(arrs_tex_names)
is_conc_fft_vv = [True, False] + [False] * len(indices) + [False] * len(arrs_tex_names)
is_conc_fft_evi = [False, False, True] + [False] * (len(indices) - 1) + [False] * len(arrs_tex_names)
is_conc_fft_none = [False, False, False] + [False] * (len(indices) - 1) + [False] * len(arrs_tex_names)

is_concerned = is_conc_fft_vv_vh_evi
is_concerned_ext = [True, True] + [True] * len(indices) + [True] * len(arrs_tex_names)

In [ ]:
pip_fft = Pipeline([('fft', FFCombo(len_inp=len(samples), is_concerned=is_concerned,
                                              scaler=std_norm, thr_freq=5, scale=True, quantile_range=(0.25, 0.75), 
                                              ordered_tkwargs=ordered_tkwargs_fft))
                   ])

In [ ]:
pip_other = Pipeline([('preproc', 
                       DynamicFeatureUnion([
                           ('min', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(np.min, axis=1))], is_on=1)),
                            
                           ('max', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(np.max, axis=1))], is_on=1)),
                           
                           ('mean', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(np.median, axis=1))], is_on=1)),
                           ('q25', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(partial(np.quantile, q=0.25), axis=1))], is_on=1)),
                           ('q75', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(partial(np.quantile, q=0.75), axis=1))], is_on=1)),
                           ('std', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(np.std, axis=1))], is_on=1)),
                           ('median', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(np.mean, axis=1))], is_on=1)),
                           ('diff_suwi_mean', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(diff_suwi, reducer=np.mean))], is_on=0)),
                           ('diff_suwi_std', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(diff_suwi, reducer=np.std))], is_on=1)),
                           ('diff_suwi_q75', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(diff_suwi, reducer=partial(np.quantile, q=0.75)))], is_on=1)),
                           ('diff_suwi_q25', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=len(samples), 
                                                                                        transformer=get_scaled_func_transformer(diff_suwi, reducer=partial(np.quantile, q=0.25)))], is_on=1))
                     ]))])


In [ ]:
preproc = Pipeline([#('input_format', FunctionTransformer(to_input_list)), 
                    ('scale', normer),
                    #('pca', TransformerSwitch(transformers=[None,
                    #                                        PCA(n_components=40)], is_on=0)),
                    ('feats', DynamicFeatureUnion([('scaled_fft', pip_fft),
                                            ('scaled_ext', pip_other)]))
                   ])

In [ ]:
pip_ext = Pipeline([('preproc', preproc),
                    ('pca', TransformerSwitch(transformers=[None, PCA(n_components=10)], is_on=False, transparent=True)), 
                    ('clustering', TransformerSwitch(transformers=[cl.KMeans(n_clusters=10, n_jobs=12),
                                                                   #cl.AffinityPropagation(max_iter=500),
                                                                   GaussianMixture(n_components=10,),
                                                                   #cl.AgglomerativeClustering(n_clusters=10, linkage='ward'),
                                                                   #cl.MeanShift(max_iter=500, n_jobs=12)
                                                                  ], is_on=0))])

In [ ]:
_ = pip_ext.fit(InputList(samples))

In [ ]:
a = pip_ext['clustering'].get().cluster_centers_
#a = pip_ext['clustering'].get().means_

inds = (pip_fft['fft'].get_feature_names() + \
              ['mean %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['mean'] is not None] + 
              ['min %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['min'] is not None] + 
              ['max %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['max'] is not None] + 
              #['median %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i]] + \
              ['q25 %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['q25'] is not None] + 
              ['q75 %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['q75'] is not None] + 
              ['std %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['std'] is not None] +
              ['diff_suwi_mean %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['diff_suwi_mean'] is not None] +
              ['diff_suwi_std %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['diff_suwi_std'] is not None] +
              ['diff_suwi_q75 %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['diff_suwi_q75'] is not None] +
              ['diff_suwi_q25 %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['diff_suwi_q25'] is not None]
       )
pd.DataFrame(a.transpose(), index=inds).hvplot(rot=90, width=1700, height=400, cmap=colormap) #cmap=['#ffffff'] + palettes.Category10[8]) #, xformatter='%.3f')#.opts(colorbar=True, cmap='paired')

In [ ]:
param_grid = {'preproc__feats__scaled_ext__preproc__median__is_on': [1],
              'preproc__feats__scaled_ext__preproc__q25__is_on': [1],
              'preproc__feats__scaled_ext__preproc__q75__is_on': [1],
              'preproc__feats__scaled_ext__preproc__min__is_on': [1],
              'preproc__feats__scaled_ext__preproc__max__is_on': [1],
              'preproc__feats__scaled_ext__preproc__std__is_on': [1],
              'preproc__feats__scaled_ext__preproc__mean__is_on': [1],
              'preproc__feats__scaled_ext__preproc__diff_suwi_mean__is_on': [1],
              'preproc__feats__scaled_ext__preproc__diff_suwi_std__is_on': [0],
              'preproc__feats__scaled_ext__preproc__diff_suwi_q25__is_on': [1],
              'preproc__feats__scaled_ext__preproc__diff_suwi_q75__is_on': [1],
              #'preproc__feats__scaled_fft__fft__thr_freq': [3, 4, 5, 6, 10],
              'preproc__feats__scaled_fft__fft__is_concerned': [is_conc_fft_vv_vh_evi],#is_conc_fft_vv_vh, ],
              #'pca__is_on': [False, True],
              #'pca__n_components': [10, 20, 40, 60],
              #'clustering__is_on': [0],
              'clustering__n_clusters': [10, 20, 30, 40, 50, 60, 100, 200]}
              #'clustering__n_components': ['--', 30, 50, 70, 100, 150, 200],}
              #'clustering__linkage': ['--', 'single', 'average', 'complete']}

In [ ]:
gs = GridSearch(pip_ext, param_grid, cv=4, scores=['adjusted_rand_score', 'adjusted_mutual_info_score', 'fowlkes_mallows_score', 'v_measure_score'])
res, params = gs.fit(InputList(samples), gts, joblib=True, n_jobs=8, verbose=50)
res, params

In [ ]:
df = pd.DataFrame(columns=list(param_grid.keys()) + list(gs.scores) + ['gt'])
for i in range(len(res)):
    for j in range(len(res[i])):
        #params[j]['preproc__feats__scaled_fft__fft__is_concerned'] = np.sum(params[j]['preproc__feats__scaled_fft__fft__is_concerned'])
        row = dict(**dict(zip(gs.scores, res[i][j]), **params[j]))
        row['gt'] = i
        #row['preproc__feats__scaled_fft__fft__is_concerned'] = 0#np.sum(row['preproc__feats__scaled_fft__fft__is_concerned'] > 0)
        df = df.append(row, ignore_index=True)
        
filter_col = [col for col in df if col.startswith('preproc__feats__scaled')]
df[filter_col] = df[filter_col].replace('--', 0)

#df['clustering__n_clusters'] = df['clustering__n_clusters'].replace('--', 10)

uniques = np.unique(df['preproc__feats__scaled_fft__fft__is_concerned'])
df['preproc__feats__scaled_fft__fft__is_concerned'] = df['preproc__feats__scaled_fft__fft__is_concerned'].apply(lambda x: [i for i, y in enumerate(uniques) if x == y][0])

In [ ]:
df.to_csv('/home/jim/PycharmProjects/seminar/notebooks/grid_search_all_gts_yearly.csv')

In [ ]:
preproc_feats_cols = [col for col in df if col.startswith('preproc__feats__scaled_ext')]

figs = []
for val in ['adjusted_mutual_info_score']:
    figs.append(df.hvplot.scatter('clustering__n_clusters', val, by=['gt']))
hv.Overlay(figs).opts(width=1200, height=500)

In [ ]:
df = pd.read_csv('/home/jim/PycharmProjects/seminar/notebooks/grid_search_all_gts_yearly.csv')
df.sort_values('adjusted_mutual_info_score', ascending=False).iloc[0]

## Summarizing everything

### Fit in total extent in 2017 (or any other year)

In [ ]:
time_series_results = {}

In [ ]:
year = None #2017
ground_truth_depth_non_wetland = 1
ground_truth_depth_wetland = 1
mask = None #pd.DataFrame(arr_wm_all.data.flatten())

In [ ]:
samples, gt_samples, sample_index = sample(100000, ground_truth=only_valid_gt, mask=mask)

if year is not None:
    samples = filter_time(InputList(samples), year=year)
else:
    samples = InputList(samples)
    
gtt_samples = reduce_gt(gt_samples, ground_truth_depth_wetland, ground_truth_depth_non_wetland)

gtr = reduce_gt(aligned_gt.data, ground_truth_depth_wetland, ground_truth_depth_non_wetland)

gtr = xr.DataArray(gtr, dims=['band', 'y', 'x'])
gtr.attrs = aligned_gt.attrs
gtr = gtr.assign_coords(aligned_gt.coords)

In [ ]:
samples, gtt_samples, sample_index = sample(100000, ground_truth=only_valid_gt_reduced, mask=mask)

if year is not None:
    samples = filter_time(InputList(samples), year=year)
else:
    samples = InputList(samples)
    
gtt_samples = gtt_samples
gtr = aligned_gt_reduced

In [ ]:
label_set, label_counts = np.unique(gtr.data, return_counts=True)

label_to_idx = dict(zip(label_set, range(len(label_set))))
label_to_idx[-1] = -1
mapper_gtr = np.vectorize(lambda ind: label_to_idx.get(ind))

In [ ]:
#is_concerned = [False, True] + [False] * len(arrs_opt_names) + [False] * len(indices)
#is_concerned_ext = [True, True] + [True, True, True] + [False]* (len(arrs_opt_names) - 3) + [True] * len(indices)
is_conc_fft_vv_vh = [True, True] + [False] * len(indices) + [False] * len(arrs_tex_names)
is_conc_fft_vv_vh_evi = [True, True] + [True, False, False, False, False, False, False] + [False] * len(arrs_tex_names)
is_conc_fft_vh = [False, True] + [False] * len(indices) + [False] * len(arrs_tex_names)
is_conc_fft_vv = [True, False] + [False] * len(indices) + [False] * len(arrs_tex_names)
is_conc_fft_evi = [False, False, True] + [False] * (len(indices) - 1) + [False] * len(arrs_tex_names)
is_conc_fft_none = [False, False, False] + [False] * (len(indices) - 1) + [False] * len(arrs_tex_names)

is_concerned = is_conc_fft_vv_vh_evi
is_concerned_ext = [True, True] + [True] * len(indices) + [True] * len(arrs_tex_names)

In [ ]:
pip_fft = Pipeline([('fft', FFCombo(len_inp=samples.nr_inputs, is_concerned=is_concerned,
                                              scaler=std_norm, thr_freq=8, scale=True, quantile_range=(0.25, 0.75), 
                                              ordered_tkwargs=ordered_tkwargs_fft))
                   ])

In [ ]:
pip_other = Pipeline([('preproc', 
                       DynamicFeatureUnion([
                           ('min', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=samples.nr_inputs, 
                                                                                        transformer=get_scaled_func_transformer(np.min, axis=1))], is_on=1)),
                            
                           ('max', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=samples.nr_inputs, 
                                                                                        transformer=get_scaled_func_transformer(np.max, axis=1))], is_on=1)),
                           
                           ('mean', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=samples.nr_inputs, 
                                                                                        transformer=get_scaled_func_transformer(np.median, axis=1))], is_on=1)),
                           ('q25', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=samples.nr_inputs, 
                                                                                        transformer=get_scaled_func_transformer(partial(np.quantile, q=0.25), axis=1))], is_on=1)),
                           ('q75', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=samples.nr_inputs, 
                                                                                        transformer=get_scaled_func_transformer(partial(np.quantile, q=0.75), axis=1))], is_on=1)),
                           ('std', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=samples.nr_inputs, 
                                                                                        transformer=get_scaled_func_transformer(np.std, axis=1))], is_on=1)),
                           ('median', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=samples.nr_inputs, 
                                                                                        transformer=get_scaled_func_transformer(np.mean, axis=1))], is_on=1)),
                           ('diff_suwi_mean', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=samples.nr_inputs, 
                                                                                        transformer=get_scaled_func_transformer(diff_suwi, reducer=np.mean))], is_on=1)),
                           ('diff_suwi_std', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=samples.nr_inputs, 
                                                                                        transformer=get_scaled_func_transformer(diff_suwi, reducer=np.std))], is_on=0)),
                           ('diff_suwi_q75', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=samples.nr_inputs, 
                                                                                        transformer=get_scaled_func_transformer(diff_suwi, reducer=partial(np.quantile, q=0.75)))], is_on=1)),
                           ('diff_suwi_q25', TransformerSwitch(transformers=[None, 
                                                                   FuncTransformerCombo(is_concerned=is_concerned_ext, len_inp=samples.nr_inputs, 
                                                                                        transformer=get_scaled_func_transformer(diff_suwi, reducer=partial(np.quantile, q=0.25)))], is_on=1))
                     ]))])


In [ ]:
preproc = Pipeline([#('input_format', FunctionTransformer(to_input_list)), 
                    ('scale', normer),
                    #('pca', TransformerSwitch(transformers=[None,
                    #                                        PCA(n_components=40)], is_on=0)),
                    ('feats', FeatureUnion([('scaled_fft', pip_fft),
                                            ('scaled_ext', pip_other)
                                           ]))
                    #('scaled_ext', pip_fft)
                   ])

In [ ]:
pip_ext = Pipeline([('preproc', preproc),
                    ('pca', TransformerSwitch(transformers=[None, PCA(n_components=10)], is_on=False, transparent=True)), 
                    ('clustering', TransformerSwitch(transformers=[cl.KMeans(n_clusters=30, n_jobs=12),
                                                                   #cl.AffinityPropagation(max_iter=500),
                                                                   GaussianMixture(n_components=30,),
                                                                   #cl.AgglomerativeClustering(n_clusters=10, linkage='ward'),
                                                                   #cl.MeanShift(max_iter=500, n_jobs=12)
                                                                   RandomForestClassifier(max_depth=5, n_estimators=100, max_features=1)
                                                                  ], is_on=-0))])

In [ ]:
pip, assign, sa_out = cluster_based_classif(samples, gtt_samples, pip_ext, n_clusters=pip_ext['clustering'].get().n_clusters, score=f1_score, train=True, freq_weighted=True,
                                    annealing_params=dict(epochs=200, T=5e-3, eta=0.99, max_change=1, verbose=True, sample=10, subset=1))

In [ ]:
plt.plot([h[1].mean() for h in sa_out['hist']])
plt.ylabel(r'mean of f1 scores')
plt.xlabel('epochs')

plt.gcf().set_figwidth(10)

In [ ]:
pip = pip_ext
a = pip['clustering'].get().cluster_centers_
#a = pip_ext['clustering'].get().means_

inds = (pip_fft['fft'].get_feature_names() + \
              ['mean %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['mean'] is not None] + 
              ['min %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['min'] is not None] + 
              ['max %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['max'] is not None] + 
              ['median %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i]] + \
              ['q25 %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['q25'] is not None] + 
              ['q75 %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['q75'] is not None] + 
              ['std %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['std'] is not None] +
              ['diff_suwi_mean %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['diff_suwi_mean'] is not None] +
              ['diff_suwi_std %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['diff_suwi_std'] is not None] +
              ['diff_suwi_q75 %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['diff_suwi_q75'] is not None] +
              ['diff_suwi_q25 %s' % n for i, n in enumerate(all_names) if is_concerned_ext[i] and dict(pip_other['preproc'].transformer_list)['diff_suwi_q25'] is not None]
       )
pd.DataFrame(a.transpose(), index=inds).hvplot(rot=90, width=1700, height=400, cmap=colormap) #cmap=['#ffffff'] + palettes.Category10[8]) #, xformatter='%.3f')#.opts(colorbar=True, cmap='paired')

In [ ]:
if mask is not None:
    inp = mask_dframe_with_dframe(InputList(only_valid_dfxx_unnorm), pd.DataFrame(arr_wm_all.data.flatten()))
else:
    inp = InputList(only_valid_dfxx_unnorm)
    
if year is not None:
    inp = filter_time(inp, year=year)

In [ ]:
clf, (valids, conf) = train_predict_conf(pip=pip['pip'], 
                                          dset=inp, 
                                          model_arr=arrs_vv[0, :1], 
                                          ground_truth=gtr,
                                          classif_out_path='/home/jim/PycharmProjects/seminar/notebooks/data/preds/all_time__all_space__200cl__gtt33',
                                          samples=samples,
                                          is_aligned=True,
                                          load_classif=False,
                                          gt=None,
                                          train=False,
                                          gt_nan=-1,
                                          pred_nan=-1, 
                                          n_jobs=8, chunks=1, verbose=10, n_batches=300, joblib=True)

clf.data = clf.data.astype(np.int)
clf.data[np.where(clf.data == 65535)] = -1

assignment_to_label = dict(zip(range(len(assign)), assign))
assignment_to_label[-1] = -1  
mapper = np.vectorize(lambda x: assignment_to_label.get(x, None))

clf = clf.copy()
clf.data = mapper(clf.data)

labels = np.unique(gtr.data)[1:]
prec, reca, f1 = precision_score(gtr.data[valids], clf.data[valids], average=None), recall_score(gtr.data[valids], clf.data[valids], average=None), f1_score(gtr.data[valids], clf.data[valids], average=None)
#time_series_results[year] = (clf, valids, conf, prec, reca, f1, labels)


In [ ]:
mask = arr_wm_all.data
    
mask = np.logical_and(np.logical_and(mask, clf.data != -1), gtr.data != -1)
gtrm = gtr.data[mask].flatten()
clfm = clf.data[mask].flatten()

f1m_all = f1_score(gtrm, clfm, average=None).copy()

In [ ]:
f1_all = f1.copy()

In [ ]:
labels = np.unique(gtr.data)[1:]

In [ ]:
#clffn_colors = 100
#colorset = list(map(str, range(10))) + ['a', 'b', 'c', 'd', 'e', 'f']
#colormap = ['#' + ''.join(map(str, np.random.choice(colorset, 6))) for i in range(n_colors)]

#cmap = {i: c for i, c in enumerate(colormap)}
#cmap[-1] = '#ffffff'

In [ ]:
#colormap[0] = '#ffe356'
#colormap[1] = '#ab1243'

In [ ]:
from matplotlib import colors
labels = np.unique(gtr.data)[1:]
labels += 300


raw_cmap = [colors.to_rgb(colors.hex2color(c)) for c in colormap]
cmap = colors.ListedColormap(['#ffffff'] + raw_cmap)
norm = colors.BoundaryNorm(np.r_[-1, labels], ncolors=len(labels))

_ = plt.matshow(np.atleast_2d(np.r_[-1, labels]), cmap=cmap, norm=norm)

_ = plt.xticks(range(len(labels) + 1), np.r_[-1, labels-300])
_ = plt.yticks([], [])

In [ ]:
labels = np.unique(gtr.data)[1:]
label_to_idx = dict(zip(labels, range(len(labels))))
label_to_idx[-1] = -1
mapper_gtr = np.vectorize(lambda ind: label_to_idx.get(ind, -1))

clff = clf.copy()
clff.data = mapper_gtr(clff.data)

uniques = np.unique(clff.data)
cmap = list(np.array(colormap)[uniques])

label_to_idx2 = dict(zip(uniques, range(len(uniques))))
mapper_gtr2 = np.vectorize(lambda ind: label_to_idx2.get(ind, -1))

clff.data = mapper_gtr2(clff.data)

In [ ]:
np.unique(clf)

In [ ]:
np.unique(clff.data)

In [ ]:
clff.hvplot.image('x', 'y', rasterize=False, height=1200, width=1200, 
                       cmap=cmap, colorbar=False)# cmap='viridis', # cmap=['#ffffff'] + palettes.Category10[9])


In [ ]:
%matplotlib inline
plt.figure(figsize=(20, 20))
plot_conf(conf, n_clusters=30, n_classes=10, labels = np.unique(gtr.data)[1:])
plt.gcf().set_figheight(17)
plt.gcf().set_figwidth(11)

### Predict Time series

In [ ]:
time_series_results_no_fft = {}
time_series_results_no_fft[2016] = clf, valids, conf, prec, reca, f1, labels, copy.deepcopy(pip)

In [ ]:
time_series_results_gt11 = {}

In [ ]:
for i, year in enumerate([2016, 2017, 2018]):
    col = np.where(pd.to_datetime(arrs_wm_yearly_max.time.data).year == year)[0]
    #inp = mask_dframe_with_dframe(InputList(only_valid_dfxx_unnorm), pd.DataFrame(arrs_wm_yearly_max[col].data.flatten()))
    samples_year = filter_time(samples, year=year)
    inp = filter_time(InputList(only_valid_dfxx_unnorm), year=year)
    
    pip, assign, sa_out = cluster_based_classif(samples_year, gtt_samples, pip_ext, n_clusters=pip_ext['clustering'].get().n_clusters, score=f1_score, train=True, freq_weighted=True,
                                    annealing_params=dict(epochs=200, T=5e-3, eta=0.99, max_change=1, verbose=True, sample=10, subset=1))

    clf, (valids, conf) = train_predict_conf(pip=pip['pip'], 
                                              dset=inp, 
                                              model_arr=arrs_vv[0, :1], 
                                              ground_truth=gtr,
                                              classif_out_path='/home/jim/PycharmProjects/seminar/notebooks/data/clustered_year_%d.tif' % year,
                                              samples=None,
                                              is_aligned=True,
                                              load_classif=False,
                                              gt=None,
                                              train=False,
                                              gt_nan=-1,
                                              pred_nan=-1, 
                                              n_jobs=8, chunks=1, verbose=10, n_batches=300, joblib=True)
    
    clf.data = clf.data.astype(np.int)
    clf.data[np.where(clf.data == 65535)] = -1

    assignment_to_label = dict(zip(range(len(assign)), assign))
    assignment_to_label[-1] = -1  
    mapper = np.vectorize(lambda x: assignment_to_label.get(x, None))

    clf = clf.copy()
    clf.data = mapper(clf.data)

    labels = np.unique(gtr.data)[1:]
    prec, reca, f1 = precision_score(gtr.data[valids], clf.data[valids], average=None), recall_score(gtr.data[valids], clf.data[valids], average=None), f1_score(gtr.data[valids], clf.data[valids], average=None)
    time_series_results_gt11[year] = (clf, valids, conf, prec, reca, f1, labels, copy.deepcopy(pip))



In [ ]:
f1m = []
f1s = []

precs = []
precm = []

recas = []
recam = []
ts = time_series_results_gt11

for i, year in enumerate([2016, 2017, 2018]):
    clf = ts[year][0].copy()
    mask = arrs_wm_yearly_min[i].data
    
    mask = np.logical_and(np.logical_and(mask, clf.data != -1), gtr.data != -1)
    gtrm = gtr.data[mask].flatten()
    clfm = clf.data[mask].flatten()
    
    f1 = f1_score(gtrm, clfm, average=None)
    f1m.append(f1)
    f1s.append(ts[year][5])
    
    prec = precision_score(gtrm, clfm, average=None)
    precm.append(prec)
    precs.append(ts[year][4])
    
    reca = recall_score(gtrm, clfm, average=None)
    recam.append(reca)
    recas.append(ts[year][3])

In [ ]:
res = pd.DataFrame()
for i, year in enumerate([2016, 2017, 2018]):
    res_year = pd.DataFrame({'OPT-YR %d MASKED' % year: f1m[i], 'OPT-YR %d UNMASKED' % year: f1s[i]})
    res = pd.concat([res, res_year], axis=1)
res = pd.concat([res, pd.DataFrame({'OPT UNMASKED': f1_all, 'OPT MASKED': f1m_all})], axis=1)

In [ ]:
#labels = np.array([1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 14])
#cols = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11]

labels = [1, 2, 3, 4, 5]

res.loc[:, 'label'] = labels
res = res.set_index('label')
res.loc[:, ['OPT-YR 2016 UNMASKED', 'OPT-YR 2017 UNMASKED', 'OPT-YR 2018 UNMASKED', 'OPT UNMASKED', 
            'OPT-YR 2016 MASKED', 'OPT-YR 2017 MASKED', 'OPT-YR 2018 MASKED', 'OPT MASKED']].hvplot.bar(width=1500, height=400, rot=90, ylim=(0, 1), ylabel='f1', xlabel='')

In [ ]:
labels = np.array([1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 14])
for i, year in enumerate([2016, 2017, 2018]):
    plt.figure(figsize=(20, 20))
    ax1 = plt.subplot(311)
    ax1.set_title('OPT-YR %d' % year)
    ax1.bar(x=labels-0.2, height=f1s[i], width=0.3, align='center', alpha=1)
    ax1.bar(x=labels+0.2, height=f1m[i], width=0.3, align='center', alpha=1)
    ax1.set_ylabel('f1')
    ax1.set_xlabel('class')
    
    ax2 = plt.subplot(312)
    ax2.set_title('OPT-YR %d' % year)
    ax2.bar(x=labels-0.2, height=precs[i], width=0.3, align='center', alpha=1)
    ax2.bar(x=labels+0.2, height=precm[i], width=0.3, align='center', alpha=1)
    ax2.set_ylabel('f1')
    ax2.set_xlabel('class')
    
    ax3 = plt.subplot(313)
    ax3.set_title('OPT-YR %d' % year)
    ax3.bar(x=labels-0.2, height=recas[i], width=0.3, align='center', alpha=1)
    ax3.bar(x=labels+0.2, height=recam[i], width=0.3, align='center', alpha=1)
    ax3.set_ylabel('f1')
    ax3.set_xlabel('class')
    

    
    plt.xticks(labels)